# Comparison of DFT solvers

We compare four different approaches for solving the DFT minimisation problem,
namely a density-based SCF, a potential-based SCF, direct minimisation and Newton.

First we setup our problem

In [1]:
using DFTK
using LinearAlgebra

a = 10.26  # Silicon lattice constant in Bohr
lattice = a / 2 * [[0 1 1.];
                   [1 0 1.];
                   [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms     = [Si, Si]
positions = [ones(3)/8, -ones(3)/8]

model = model_LDA(lattice, atoms, positions)
basis = PlaneWaveBasis(model; Ecut=5, kgrid=[3, 3, 3])

# Convergence we desire
tol = 1e-12
is_converged = DFTK.ScfConvergenceDensity(tol);

## Density-based self-consistent field

In [2]:
scfres_scf = self_consistent_field(basis; is_converged);

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -7.846884282458                   -0.70    4.8
  2   -7.852304965042       -2.27       -1.54    1.0
  3   -7.852646212946       -3.47       -2.52    3.2
  4   -7.852646676505       -6.33       -3.37    2.2
  5   -7.852646686174       -8.01       -4.66    1.8
  6   -7.852646686728       -9.26       -5.39    3.2
  7   -7.852646686730      -11.71       -6.19    1.0
  8   -7.852646686730      -12.96       -7.87    2.2
  9   -7.852646686730   +    -Inf       -7.99    3.2
 10   -7.852646686730   +    -Inf       -9.25    2.5
 11   -7.852646686730   +    -Inf       -8.85    1.0
 12   -7.852646686730   +  -15.05      -10.25    2.2
 13   -7.852646686730   +  -14.57      -10.93    2.8
 14   -7.852646686730      -14.45      -11.55    1.8
 15   -7.852646686730   +    -Inf      -12.28    3.0


## Potential-based SCF

In [3]:
scfres_scfv = DFTK.scf_potential_mixing(basis; is_converged);

n     Energy            log10(ΔE)   log10(Δρ)   α      Diag
---   ---------------   ---------   ---------   ----   ----
  1   -7.846579844340                   -0.70           4.2
  2   -7.852523331279       -2.23       -1.64   0.80    2.5
  3   -7.852635530567       -3.95       -2.72   0.80    1.0
  4   -7.852646455300       -4.96       -3.23   0.80    2.0
  5   -7.852646667395       -6.67       -4.07   0.80    1.0
  6   -7.852646686430       -7.72       -4.79   0.80    2.0
  7   -7.852646686722       -9.53       -5.53   0.80    1.8
  8   -7.852646686729      -11.14       -6.69   0.80    1.8
  9   -7.852646686730      -12.32       -7.13   0.80    2.8
 10   -7.852646686730      -14.35       -8.71   0.80    1.0
 11   -7.852646686730   +  -14.57       -8.38   0.80    3.2
 12   -7.852646686730      -14.57       -9.51   0.80    1.0
 13   -7.852646686730   +    -Inf       -9.60   0.80    2.0
 14   -7.852646686730   +  -14.75      -10.98   0.80    1.0
 15   -7.852646686730      -14.35      -

## Direct minimization

In [4]:
scfres_dm = direct_minimization(basis; tol);

Iter     Function value   Gradient norm 
     0     1.425381e+01     3.385699e+00
 * time: 0.45820188522338867
     1     1.871260e+00     2.063258e+00
 * time: 0.7304139137268066
     2    -1.429179e+00     2.372134e+00
 * time: 0.7595748901367188
     3    -3.594853e+00     1.880530e+00
 * time: 0.8001198768615723
     4    -5.099509e+00     1.412771e+00
 * time: 0.8400189876556396
     5    -6.775656e+00     9.380122e-01
 * time: 0.8802459239959717
     6    -6.803340e+00     1.452442e+00
 * time: 0.9071497917175293
     7    -7.445653e+00     9.671171e-01
 * time: 0.9350588321685791
     8    -7.676529e+00     3.340431e-01
 * time: 0.9745209217071533
     9    -7.751657e+00     1.675841e-01
 * time: 1.003173828125
    10    -7.792868e+00     1.070677e-01
 * time: 1.0313518047332764
    11    -7.819797e+00     1.352720e-01
 * time: 1.0589849948883057
    12    -7.836127e+00     6.348034e-02
 * time: 1.086167812347412
    13    -7.847085e+00     3.659400e-02
 * time: 1.11468100547790

## Newton algorithm

Start not too far from the solution to ensure convergence:
We run first a very crude SCF to get close and then switch to Newton.

In [5]:
scfres_start = self_consistent_field(basis; tol=1e-1);

n     Energy            log10(ΔE)   log10(Δρ)   Diag
---   ---------------   ---------   ---------   ----
  1   -7.846870286635                   -0.70    4.8
  2   -7.852328417337       -2.26       -1.53    1.0


Remove the virtual orbitals (which Newton cannot treat yet)

In [6]:
ψ = DFTK.select_occupied_orbitals(basis, scfres_start.ψ, scfres_start.occupation).ψ
scfres_newton = newton(basis, ψ; tol);

n     Energy            log10(ΔE)   log10(Δρ)
---   ---------------   ---------   ---------
  1   -7.852646686712                   -2.55
  2   -7.852646686730      -10.74       -5.99
  3   -7.852646686730   +    -Inf      -12.64


## Comparison of results

In [7]:
println("|ρ_newton - ρ_scf|  = ", norm(scfres_newton.ρ - scfres_scf.ρ))
println("|ρ_newton - ρ_scfv| = ", norm(scfres_newton.ρ - scfres_scfv.ρ))
println("|ρ_newton - ρ_dm|   = ", norm(scfres_newton.ρ - scfres_dm.ρ))

|ρ_newton - ρ_scf|  = 2.39284107024252e-13
|ρ_newton - ρ_scfv| = 4.292818335793729e-13
|ρ_newton - ρ_dm|   = 2.582211720960404e-10
